# Get Sample Colors
> Get sample colors by using [K-Means Clustering](https://en.wikipedia.org/wiki/K-means_clustering)

[Back to <b>contents</b>](../README.md)

First, represent the main colors of shoe in shoes.csv in 3D. <br>
Numpy, Matplotlib and csv libraries are needed

In [ ]:
# reference : https://engineer-mole.tistory.com/250
import numpy as np
import matplotlib.pyplot as plt
import csv
def load_shoe_data(filename):
  class ShoeData:
    data = []
    target = []
    feature_names = ['Name','Brand','Colors','Link','wish','review']
  shoes = ShoeData()
  with open(filename,'r', encoding ='utf-8') as f:
    for items in csv.reader(f):
      if items[0].startswith('#'): continue
      shoes.target.append(eval(items[2]))
      shoes.data.append(items)
    shoes.data = np.array(shoes.data)
    return shoes

shoes = load_shoe_data('data/shoes.csv')
fig = plt.figure(figsize = (8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.set_title("Main Colors of Shoes (view_init(30, 330))", size = 20)
ax.set_xlabel("R", size = 14)
ax.set_ylabel("G", size = 14)
ax.set_zlabel("B", size = 14)
for colors in shoes.target:
  RGBP = colors[0]
  RGB = np.array(RGBP[:-1])/255.0
  ax.scatter(RGB[0], RGB[1], RGB[2], color = RGB)
ax.view_init(30, 330)
plt.show()

Each view_init show plot by different angles.

<img src = "../img/MCS.png" width = 800vw>

### Data analysis
<div style ="backgroud-color : ">
1. As expected, the main colors of shoe are mostly achromatic.
2. <span style ="background-color : Black; color : white;">Black</span> and <span style ="background-color : White; color : Black;">white</span> are usually abundant, and the rest of the colors are evenly distributed.
</div>


## Goal
> I want to compress <b>color information</b> into <b>less number of colors</b>.
1. The above plots include only the main colors of shoe (1000 colors), so <span style = "color : red;">all colors of shoes are not used.</span>
2. A small number of colors should <mark>effeciently</mark> represent the color of the shoes.
